This notebook is a driver program for \_______ _(Insert name here)_ library

In [7]:
import taxes.download as t

In [10]:
# getting all video links 
sheets = t.get_sheet_links_names(2019) 

t.download_sheet_series(sheets)

print(sheets)

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Gminy_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Powiaty_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Miasta_NPP_za_2019.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Gornoslasko_Zaglebiowska_Metropolia.xlsx downloaded!

/Users/k4cp3rskiii/Library/Mobile Documents/com~apple~CloudDocs/MISMaP/NPD_Final/NPD_final/data
20200214_Wojewodztwa_za_2019.xlsx downloaded!

All sheets downloaded!
(['https://www.gov.pl/attachment/6594af94-cd1e-49fb-9149-99fd663aef25', 'https://www.gov.pl/attachment/5f2abc44-6a7e-4b73-8999-696920252efc', 'https://www.gov.pl/attachment/141da745-800d-44c5-ac97-e90c4cbd5e11', 'https://www.gov.pl/attachment/12150aff-d70e-